In [0]:
model_coordinates = 'users.antoine_amend.slack-bot-rag'

In [0]:
import pandas as pd

history = [
  ['HUMAN', 'Hello, how are you?'],
  ['BOT', 'I am doing great, what about yourself?'],
  ['HUMAN', 'Thanks for asking. Can you help me with a question?'],
  ['BOT', 'Sure, what\s up?'],
  ['HUMAN', 'How to create a slack bot?']
]

model_input = pd.DataFrame(history, columns=['user', 'text'])

In [0]:
import mlflow
from mlflow.pyfunc import PythonModel

class DummySlackModel(mlflow.pyfunc.PythonModel):

  def predict(self, context, model_input):
    import pandas as pd
    history = model_input.values.tolist()
    question = history.pop(-1)[1]
    return pd.DataFrame({
      'answer': 'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.',
      'links': [
        'https://www.databricks.com/',
        'https://github.com/databricks-industry-solutions/slack-bot-rag'
      ]
    })

In [0]:
python_model = DummySlackModel()
model_output = python_model.predict(None, model_input)

In [0]:
from mlflow.models.signature import infer_signature
model_signature = infer_signature(model_input=model_input, model_output=model_output)
model_signature

inputs: 
  ['user': string, 'text': string]
outputs: 
  ['answer': string, 'links': string]
params: 
  None

In [0]:
with mlflow.start_run() as run:
  mlflow.pyfunc.log_model(
    artifact_path = 'model',
    python_model = python_model,
    signature=model_signature,
    input_example=model_input
  )

Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
from mlflow.tracking import MlflowClient
mlflow.set_registry_uri('databricks-uc')
client = MlflowClient()
latest_model = mlflow.register_model(f'runs:/{run.info.run_id}/model', model_coordinates)
client.set_registered_model_alias(model_coordinates, 'production', latest_model.version)

Successfully registered model 'users.antoine_amend.slack-bot-rag'.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

2023/11/23 17:42:45 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: users.antoine_amend.slack-bot-rag, version 1
Created version '1' of model 'users.antoine_amend.slack-bot-rag'.
